# 目的

講義第５回、第６回で扱った内容に関し、実習を通して理解を深めます。

# 課題

以下の3点を実装して実行してください

1. RLearnerの実装が間違っています。直してください。
2. XLearnerの特性を記述し、その特性が特に有効に機能するようなデータ生成過程（DGP）をDGP-Dとして実装してください
  * 関数形だけでなく、サンプルサイズや次元数を変えることも可とします
3. そのほかに1つの手法を取り上げ、その手法の特性と、それが勝てる、または意外にも負ける（S/T以外の比較的高度な手法であるにもかかわらずこれらに負ける）DGPを考えて実装してください

提出は.txtファイルで、以下を含めてください。

* DGPの作成に関しては、その狙いを簡潔に記載
  * 取り上げた手法が実際に狙った精度を出しているかどうかよりも、手法の特性と記載された狙いが合致しているかを重視します。
* 変更した部分のコード
  * 変更したコードが多い場合、全体を別ファイルとして添付することも可。その場合zipで固めてください。
* 実行結果

酷似した内容の提出が複数あった場合は無効とする可能性がありますので他人と共有しないでください。

In [ ]:
# Install section (uncomment if needed in Colab)
!pip -q install numpy scikit-learn matplotlib
!pip -q install econml  # optional (for *_EML and CausalForest)

In [ ]:
# ================== Colab CATE Assignment Script ==================
from __future__ import annotations

import numpy as np
from dataclasses import dataclass
from typing import List, Dict, Optional

# sklearn
from sklearn.ensemble import (
    GradientBoostingRegressor,
    GradientBoostingClassifier,
    RandomForestRegressor,
    RandomForestClassifier,
)
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.model_selection import train_test_split, KFold
from sklearn.base import clone

# Optional backends
try:
    from econml.dml import CausalForestDML  # optional
    _ECONML_CF_OK = True
except Exception:
    _ECONML_CF_OK = False




# ----------------------- Utils -----------------------
def _sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))


def clip01(a, lo=1e-3, hi=1 - 1e-3):
    return np.clip(a, lo, hi)


def make_regressor(kind: str, random_state: int):
    k = (kind or "").lower()
    if k in ("gbm", "gbrt", "gboost"):
        return GradientBoostingRegressor(random_state=random_state, max_depth=3)
    if k in ("rf", "randomforest", "random_forest"):
        return RandomForestRegressor(n_estimators=300, random_state=random_state, n_jobs=-1)
    if k in ("linear", "ols"):
        return LinearRegression()
    if k in ("ridge",):
        return Ridge(alpha=1.0)
    return GradientBoostingRegressor(random_state=random_state, max_depth=3)


def make_classifier(kind: str, random_state: int):
    k = (kind or "").lower()
    if k in ("logit", "lr", "logreg", "logistic"):
        return LogisticRegression(max_iter=500, solver="lbfgs")
    if k in ("rf", "randomforest", "random_forest"):
        return RandomForestClassifier(n_estimators=300, random_state=random_state, n_jobs=-1)
    if k in ("gbm", "gbrt", "gboost"):
        return GradientBoostingClassifier(random_state=random_state, max_depth=3)
    return LogisticRegression(max_iter=500, solver="lbfgs")


# ----------------------- DGPs -----------------------
@dataclass
class DGPResult:
    X: np.ndarray
    T: np.ndarray
    Y: np.ndarray
    tau_true: np.ndarray
    e_true: np.ndarray
    mu0_true: np.ndarray
    mu1_true: np.ndarray


def make_dgp(n: int, p: int, seed: int, kind: str = "A", noise_std: float = 1.0) -> DGPResult:
    rng = np.random.default_rng(seed)
    X = rng.normal(size=(n, p))

    if kind.upper() == "A":
        # RCT・線形/弱非線形・十分なオーバーラップ
        e = np.full(n, 0.5)  # mu
        beta = rng.normal(size=p) / np.sqrt(p)
        tau = X[:, 0] - X[:, 1]
        mu0 = X @ beta + 0.5 * np.tanh(X[:, 2])
        mu1 = mu0 + tau
    elif kind.upper() == "B":
        # 交絡あり・非線形・中程度オーバーラップ
        raw = 1.0 * X[:, 0] - 1.2 * X[:, 1] + 0.8 * X[:, 2]
        e = np.clip(_sigmoid(raw), 0.1, 0.9)
        tau = 0.5 * np.sin(X[:, 3]) + 0.5 * (X[:, 4] > 0).astype(float)
        mu0 = 1.0 * np.sin(X[:, 0]) + 0.5 * (X[:, 1] ** 2) + 0.25 * X[:, 2] * X[:, 3]
        mu1 = mu0 + tau
    elif kind.upper() == "C":
        # 強い不均衡（treated ~ 5-20%）
        raw = -2.0 + 1.0 * X[:, 0] - 0.5 * X[:, 2]
        e = np.clip(_sigmoid(raw), 0.02, 0.20)
        tau = 0.8 * X[:, 1]
        mu0 = 1.2 * np.sin(X[:, 0]) + 0.8 * (X[:, 1] ** 2) + 0.5 * X[:, 2] * X[:, 3] + 0.3 * np.maximum(X[:, 4], 0.0)
        mu1 = mu0 + tau
    elif kind.upper() == "D":
        # TODO ここを編集して提出してください
        pass
    else:
        raise ValueError(f"Unknown DGP kind: {kind}")

    T = rng.binomial(1, e)
    Y = mu0 * (1 - T) + mu1 * T + rng.normal(scale=noise_std, size=n)
    return DGPResult(X=X, T=T.astype(float), Y=Y, tau_true=tau, e_true=e, mu0_true=mu0, mu1_true=mu1)


# ----------------------- Learners -----------------------
class BaseCATE:
    def fit(self, X, T, Y):
        raise NotImplementedError

    def predict_tau(self, X):
        raise NotImplementedError


class SLearner(BaseCATE):
    def __init__(self, rs=0, base_reg="gbm"):
        self.m = make_regressor(base_reg, rs)

    def fit(self, X, T, Y):
        XT = np.column_stack([X, T.reshape(-1, 1)])
        self.m.fit(XT, Y)
        return self

    def predict_tau(self, X):
        n = X.shape[0]
        y1 = self.m.predict(np.column_stack([X, np.ones((n, 1))]))
        y0 = self.m.predict(np.column_stack([X, np.zeros((n, 1))]))
        return y1 - y0


class TLearner(BaseCATE):
    def __init__(self, rs=0, base_reg="gbm"):
        self.mu0 = make_regressor(base_reg, rs)
        self.mu1 = make_regressor(base_reg, rs + 1)

    def fit(self, X, T, Y):
        I0 = (T == 0)
        I1 = (T == 1)
        self.mu0.fit(X[I0], Y[I0])
        self.mu1.fit(X[I1], Y[I1])
        return self

    def predict_tau(self, X):
        return self.mu1.predict(X) - self.mu0.predict(X)


class XLearner(BaseCATE):
    def __init__(self, rs=0, base_reg="gbm", base_clf="logit", final_reg="same", cv=1):
        self.rs = rs
        self.base_reg = base_reg
        self.base_clf = base_clf
        self.final_reg = final_reg
        self.cv = max(1, int(cv))
        self.mu0 = make_regressor(base_reg, rs)
        self.mu1 = make_regressor(base_reg, rs + 1)
        self.g = make_classifier(base_clf, rs)
        f = final_reg if final_reg != "same" else base_reg
        self.tau0 = make_regressor(f, rs + 2)
        self.tau1 = make_regressor(f, rs + 3)

    def _oof_mu(self, X, T, Y):
        n = X.shape[0]
        mu0_hat = np.zeros(n)
        mu1_hat = np.zeros(n)
        e_hat = np.zeros(n)
        kf = KFold(n_splits=self.cv, shuffle=True, random_state=self.rs)
        for tr, te in kf.split(X):
            Xtr, Xte = X[tr], X[te]
            Ttr, Ytr = T[tr], Y[tr]
            mu0 = clone(make_regressor(self.base_reg, self.rs))
            mu1 = clone(make_regressor(self.base_reg, self.rs + 1))
            g = clone(make_classifier(self.base_clf, self.rs))
            I0 = (Ttr == 0)
            I1 = (Ttr == 1)
            mu0.fit(Xtr[I0], Ytr[I0]) if np.any(I0) else mu0.fit(Xtr, Ytr)
            mu1.fit(Xtr[I1], Ytr[I1]) if np.any(I1) else mu1.fit(Xtr, Ytr)
            g.fit(Xtr, Ttr)
            mu0_hat[te] = mu0.predict(Xte)
            mu1_hat[te] = mu1.predict(Xte)
            e_hat[te] = clip01(g.predict_proba(Xte)[:, 1])
        return mu0_hat, mu1_hat, e_hat

    def fit(self, X, T, Y):
        I0 = (T == 0)
        I1 = (T == 1)
        if self.cv > 1:
            mu0_hat, mu1_hat, _ = self._oof_mu(X, T, Y)
            D1 = Y[I1] - mu0_hat[I1]
            D0 = mu1_hat[I0] - Y[I0]
            self.g.fit(X, T)
            self.tau1.fit(X[I1], D1)
            self.tau0.fit(X[I0], D0)
        else:
            self.mu0.fit(X[I0], Y[I0])
            self.mu1.fit(X[I1], Y[I1])
            D1 = Y[I1] - self.mu0.predict(X[I1])
            D0 = self.mu1.predict(X[I0]) - Y[I0]
            self.g.fit(X, T)
            self.tau1.fit(X[I1], D1)
            self.tau0.fit(X[I0], D0)
        return self

    def predict_tau(self, X):
        g = clip01(self.g.predict_proba(X)[:, 1])
        return g * self.tau0.predict(X) + (1 - g) * self.tau1.predict(X)


class DRLearner(BaseCATE):
    def __init__(self, rs=0, base_reg="gbm", base_clf="logit", final_reg="same", cv=1):
        self.rs = rs
        self.base_reg = base_reg
        self.base_clf = base_clf
        self.final_reg = final_reg
        self.cv = max(1, int(cv))
        self.mu0 = make_regressor(base_reg, rs)
        self.mu1 = make_regressor(base_reg, rs + 1)
        self.g = make_classifier(base_clf, rs)
        f = final_reg if final_reg != "same" else base_reg
        self.tau = make_regressor(f, rs + 2)

    def _oof(self, X, T, Y):
        n = X.shape[0]
        mu0_hat = np.zeros(n)
        mu1_hat = np.zeros(n)
        e_hat = np.zeros(n)
        kf = KFold(n_splits=self.cv, shuffle=True, random_state=self.rs)
        for tr, te in kf.split(X):
            Xtr, Xte = X[tr], X[te]
            Ttr, Ytr = T[tr], Y[tr]
            mu0 = clone(make_regressor(self.base_reg, self.rs))
            mu1 = clone(make_regressor(self.base_reg, self.rs + 1))
            g = clone(make_classifier(self.base_clf, self.rs))
            I0 = (Ttr == 0)
            I1 = (Ttr == 1)
            mu0.fit(Xtr[I0], Ytr[I0]) if np.any(I0) else mu0.fit(Xtr, Ytr)
            mu1.fit(Xtr[I1], Ytr[I1]) if np.any(I1) else mu1.fit(Xtr, Ytr)
            g.fit(Xtr, Ttr)
            mu0_hat[te] = mu0.predict(Xte)
            mu1_hat[te] = mu1.predict(Xte)
            e_hat[te] = clip01(g.predict_proba(Xte)[:, 1])
        return mu0_hat, mu1_hat, e_hat

    def fit(self, X, T, Y):
        if self.cv > 1:
            mu0_hat, mu1_hat, e_hat = self._oof(X, T, Y)
        else:
            self.mu0.fit(X[T == 0], Y[T == 0])
            self.mu1.fit(X[T == 1], Y[T == 1])
            self.g.fit(X, T)
            mu0_hat = self.mu0.predict(X)
            mu1_hat = self.mu1.predict(X)
            e_hat = clip01(self.g.predict_proba(X)[:, 1])
        z = ((Y - mu0_hat) * (1 - T) / (1 - e_hat)) - ((Y - mu1_hat) * T / e_hat) + (mu1_hat - mu0_hat)
        self.tau.fit(X, z)
        return self

    def predict_tau(self, X):
        return self.tau.predict(X)


class RLearner(BaseCATE):
    def __init__(self, rs=0, base_reg="gbm", base_clf="logit", final_reg="same", cv=1):
        self.rs = rs
        self.base_reg = base_reg
        self.base_clf = base_clf
        self.final_reg = final_reg
        self.cv = max(1, int(cv))
        self.m = make_regressor(base_reg, rs)
        self.g = make_classifier(base_clf, rs)
        f = final_reg if final_reg != "same" else base_reg
        self.tau = make_regressor(f, rs + 1)

    def _oof(self, X, T, Y):
        n = X.shape[0]
        m_hat = np.zeros(n)
        e_hat = np.zeros(n)
        kf = KFold(n_splits=self.cv, shuffle=True, random_state=self.rs)
        for tr, te in kf.split(X):
            Xtr, Xte = X[tr], X[te]
            Ttr, Ytr = T[tr], Y[tr]
            m = clone(make_regressor(self.base_reg, self.rs))
            g = clone(make_classifier(self.base_clf, self.rs))
            Xtr_aug = np.column_stack([Xtr, Ttr.reshape(-1, 1)])
            m.fit(Xtr_aug, Ytr)
            g.fit(Xtr, Ttr)
            Xte_aug = np.column_stack([Xte, T[te].reshape(-1, 1)])
            m_hat[te] = m.predict(Xte_aug)
            e_hat[te] = clip01(g.predict_proba(Xte)[:, 1])
        return m_hat, e_hat

    def fit(self, X, T, Y):
        if self.cv > 1:
            m_hat, e_hat = self._oof(X, T, Y)
        else:
            XT = np.column_stack([X, T.reshape(-1, 1)])
            self.m.fit(XT, Y)
            self.g.fit(X, T)
            m_hat = self.m.predict(XT)
            e_hat = clip01(self.g.predict_proba(X)[:, 1])
        y_tilde = Y - m_hat
        t_tilde = T - e_hat
        eps = 1e-3
        z = y_tilde / (t_tilde + np.sign(t_tilde) * eps)
        w = t_tilde ** 2 + 1e-6
        self.tau.fit(X, z, sample_weight=w)
        return self

    def predict_tau(self, X):
        return self.tau.predict(X)


# ----------------------- econml wrappers (optional) -----------------------
def _filter_kwargs(cls, kwargs: Dict[str, object]) -> Dict[str, object]:
    import inspect
    try:
        sig = inspect.signature(cls.__init__)
        allowed = set(sig.parameters.keys())
        allowed.discard("self")
        return {k: v for k, v in kwargs.items() if k in allowed}
    except Exception:
        return kwargs


def make_eml(name_u: str, rs: int, base_reg: str, base_clf: str, final_reg: str, cv: int) -> Optional[BaseCATE]:
    try:
        if name_u == "S_EML":
            from econml.metalearners import SLearner as E_S
            model = E_S(**_filter_kwargs(E_S, dict(overall_model=make_regressor(base_reg, rs))))
            return _EML_Effect(model)
        if name_u == "T_EML":
            from econml.metalearners import TLearner as E_T
            model = E_T(**_filter_kwargs(E_T, dict(models=make_regressor(base_reg, rs))))
            return _EML_Effect(model)
        if name_u == "X_EML":
            from econml.metalearners import XLearner as E_X
            f = final_reg if final_reg != "same" else base_reg
            kwargs = dict(
                models=[make_regressor(base_reg, rs), make_regressor(base_reg, rs + 1)],
                cate_models=[make_regressor(f, rs + 2), make_regressor(f, rs + 3)],
                propensity_model=make_classifier(base_clf, rs),
            )
            model = E_X(**_filter_kwargs(E_X, kwargs))
            return _EML_Effect(model)
        if name_u == "DR_EML":
            from econml.dr import DRLearner as E_DR
            f = final_reg if final_reg != "same" else base_reg
            kwargs = dict(
                model_regression=make_regressor(base_reg, rs),
                model_propensity=make_classifier(base_clf, rs),
                model_final=make_regressor(f, rs + 1),
                cv=cv if cv > 1 else None,
            )
            model = E_DR(**_filter_kwargs(E_DR, kwargs))
            return _EML_Effect(model)
    except Exception:
        return None
    return None


class _EML_Effect(BaseCATE):
    def __init__(self, m):
        self.m = m
    def fit(self, X, T, Y):
        self.m.fit(Y, T, X=X); return self
    def predict_tau(self, X):
        return self.m.effect(X)


# ----------------------- Additional models (CF) -----------------------
class CausalForest(BaseCATE):
    """Wrapper for econml's CausalForestDML (if available)."""
    def __init__(self, rs=0, n_estimators=400, min_samples_leaf=5):
        if not _ECONML_CF_OK:
            raise RuntimeError("econml未導入のためCausalForestは使用不可です")
        self.model = CausalForestDML(
            n_estimators=n_estimators,
            min_samples_leaf=min_samples_leaf,
            random_state=rs,
        )

    def fit(self, X, T, Y):
        self.model.fit(Y, T, X=X)
        return self

    def predict_tau(self, X):
        return self.model.effect(X)


class BARTTLearner(BaseCATE):
    """T-learner using bartpy (if available) for mu0/mu1."""
    def __init__(self, rs=0):
        if not _BARTPY_OK:
            raise RuntimeError("bartpy未導入のためBARTは使用不可です")
        # Use defaults; users can adjust by editing below if needed
        self.mu0 = BartPyRegressor()
        self.mu1 = BartPyRegressor()

    def fit(self, X, T, Y):
        I0 = (T == 0)
        I1 = (T == 1)
        self.mu0.fit(X[I0], Y[I0])
        self.mu1.fit(X[I1], Y[I1])
        return self

    def predict_tau(self, X):
        return self.mu1.predict(X) - self.mu0.predict(X)


# ----------------------- Metrics -----------------------
def pehe(y_pred, y_true):
    return float(np.sqrt(np.mean((y_pred - y_true) ** 2)))


def mae(y_pred, y_true):
    return float(np.mean(np.abs(y_pred - y_true)))


def sign_acc(y_pred, y_true):
    return float(np.mean(np.sign(y_pred) == np.sign(y_true)))


# ----------------------- Runner -----------------------
def run_once(
    dgp: str,
    n: int,
    p: int,
    seed: int,
    methods: List[str],
    base_reg: str = "gbm",
    base_clf: str = "logit",
    final_reg: str = "same",
    cv: int = 1,
):
    data = make_dgp(n=n, p=p, seed=seed, kind=dgp)
    Xtr, Xte, Ttr, Tte, Ytr, Yte, tautr, taute = train_test_split(
        data.X, data.T, data.Y, data.tau_true, test_size=0.3, random_state=seed
    )

    results = []
    for name in methods:
        u = name.upper()
        if u == "S":
            m = SLearner(rs=seed, base_reg=base_reg)
        elif u == "T":
            m = TLearner(rs=seed, base_reg=base_reg)
        elif u == "X":
            m = XLearner(rs=seed, base_reg=base_reg, base_clf=base_clf, final_reg=final_reg, cv=cv)
        elif u in ("DR", "DR-LEARNER"):
            m = DRLearner(rs=seed, base_reg=base_reg, base_clf=base_clf, final_reg=final_reg, cv=cv)
        elif u in ("R", "R-LEARNER"):
            m = RLearner(rs=seed, base_reg=base_reg, base_clf=base_clf, final_reg=final_reg, cv=cv)
        elif u == "CF":
            if _ECONML_CF_OK:
                m = CausalForest(rs=seed)
            else:
                results.append({"dgp": dgp, "method": u, "seed": seed, "note": "econml未導入: skip"});
                continue

        elif u in ("S_EML", "T_EML", "X_EML", "DR_EML"):
            m = make_eml(u, seed, base_reg, base_clf, final_reg, cv)
            if m is None:
                results.append({"dgp": dgp, "method": u, "seed": seed, "note": "econml未導入: skip"})
                continue
        else:
            raise ValueError(f"Unknown method: {name}")

        m.fit(Xtr, Ttr, Ytr)
        tau_hat = m.predict_tau(Xte)
        results.append(
            {
                "dgp": dgp,
                "method": u,
                "seed": seed,
                "n": n,
                "p": p,
                "pehe": pehe(tau_hat, taute),
                "mae_tau": mae(tau_hat, taute),
                "sign_acc": sign_acc(tau_hat, taute),
            }
        )

    return results


def summarize(rows: List[Dict[str, float]]):
    from collections import defaultdict
    agg = defaultdict(list)
    for r in rows:
        if "pehe" in r:
            agg[(r["dgp"], r["method"])].append(float(r["pehe"]))
    dgps = sorted({k[0] for k in agg.keys()})
    for d in dgps:
        print(f"\nDGP {d}:")
        stats = []
        for (dd, m), vals in agg.items():
            if dd != d:
                continue
            v = np.asarray(vals, dtype=float)
            mean = float(np.mean(v))
            se = float(np.std(v, ddof=1) / np.sqrt(len(v))) if len(v) > 1 else 0.0
            stats.append((m, mean, se, len(v)))
        for m, mean, se, n in sorted(stats, key=lambda x: x[1]):
            print(f"  {m:>8s}: PEHE={mean:.3f} ± {se:.3f} (n={n})")

# ----------------------- execute -----------------------
if __name__ == "__main__":
    repeats = 5
    base_seed = 42
    rows = []
    for i in range(repeats):
        s = base_seed + i
        for d in ["A", "B", "C"]:
            rows += run_once(
                dgp=d,
                n=800,
                p=8,
                seed=s,
                methods=["S", "T", "X", "DR", "R", "CF"],
                base_reg="rf",
                base_clf="rf",
                final_reg="linear",
                cv=3,
            )
    summarize(rows)




DGP A:
         X: PEHE=0.332 ± 0.024 (n=5)
         R: PEHE=0.492 ± 0.084 (n=5)
         T: PEHE=0.682 ± 0.037 (n=5)
        DR: PEHE=0.880 ± 0.127 (n=5)
         S: PEHE=1.011 ± 0.093 (n=5)

DGP B:
         R: PEHE=0.401 ± 0.034 (n=5)
         S: PEHE=0.427 ± 0.016 (n=5)
         X: PEHE=0.501 ± 0.027 (n=5)
        DR: PEHE=0.608 ± 0.072 (n=5)
         T: PEHE=0.708 ± 0.032 (n=5)

DGP C:
         X: PEHE=0.549 ± 0.051 (n=5)
         R: PEHE=0.610 ± 0.092 (n=5)
         S: PEHE=0.733 ± 0.020 (n=5)
         T: PEHE=0.921 ± 0.088 (n=5)
        DR: PEHE=1.467 ± 0.259 (n=5)
